# Загрузка данных с кагла

Перед этим необходимо скопировать ключ со своего акка

In [16]:
!pip install -q kaggle

from google.colab import files
files.upload()


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c hse-rec-sys-challenge-2024

!unzip hse-rec-sys-challenge-2024.zip

Saving kaggle.json to kaggle (1).json
hse-rec-sys-challenge-2024.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  hse-rec-sys-challenge-2024.zip
replace events.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: events.csv              
replace item_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: item_features.csv       
replace submission_sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_sample.csv   
replace user_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: user_features.csv       


Ставим пакеты

In [17]:
!pip install lightgbm
!pip install catboost
!pip install implicit
!pip install optuna

В нашем случае будем использовать ALS - как baseline

Catboost и LGBM - наши основные модели

Код разделил на несколько ячеек где реализовал все необходимые функции для работы. Какого-то строгого порядка нет, точнее он был, но из-за постоянной редакции очередность функций сбилась. Это не особо важно, но красоты стало меньше

In [18]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from scipy.sparse import csr_matrix
from typing import Tuple
from sklearn.preprocessing import LabelEncoder
from implicit.als import AlternatingLeastSquares
from lightgbm import LGBMClassifier
import optuna

# Загрузка данных
events = pd.read_csv('events.csv')
user = pd.read_csv('user_features.csv')
item = pd.read_csv('item_features.csv')

# Преобразование временных данных в числовой формат
def convert_timedelta_to_numeric(df):
    for col in df.select_dtypes(include=['timedelta64[ns]']).columns:
        df[col] = df[col].dt.total_seconds()  # Преобразование в секунды
    return df

# Функция для оптимизации гиперпараметров с использованием Optuna
def optimize_lightgbm(X_train, y_train):
    def objective(trial):
        # Гиперпараметры, которые будут оптимизироваться
        param = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 4, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        }

        # Обучение модели с текущими параметрами
        model = LGBMClassifier(**param)
        model.fit(X_train, y_train)
        # Оценка точности модели (например, по метрике AUC)
        score = model.score(X_train, y_train)
        return score

    # Создание и запуск исследования Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)

    return study.best_params

# LightGBM model
def fit_lightgbm(
    candidates: pd.DataFrame,
    test_events: pd.DataFrame,
    events: pd.DataFrame
) -> LGBMClassifier:
    print('Labeling candidates for LightGBM')
    candidates_labeled = (
        candidates
        .merge(
            (
                test_events[['user_id', 'item_id']]
                .assign(label=1)
            ),
            on=['user_id', 'item_id'],
            how='left',
        )
    )

    candidates_labeled['label'] = (
        candidates_labeled['label']
        .fillna(0)
        .astype('int32')
    )

    positive_classes_rate = candidates_labeled['label'].sum() / len(candidates_labeled)
    print(f'Positive classes for LightGBM = {positive_classes_rate * 100:.2f}%')

    print('Extracting features for candidates for LightGBM')
    lgb_features = enrich_interactions(candidates_labeled, events)
    del candidates_labeled

    # Преобразование временных данных в числовой формат
    lgb_features = convert_timedelta_to_numeric(lgb_features)

    X_train = lgb_features.drop(columns=['user_id', 'item_id', 'label'])
    for col in X_train.columns:
        if X_train[col].dtype == 'object':
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
            X_train[col].fillna(0, inplace=True)

    y_train = lgb_features['label']
    del lgb_features

    print('Training LightGBM')
    best_params = optimize_lightgbm(X_train, y_train)
    best_params['class_weight'] = {0: positive_classes_rate, 1: 1 - positive_classes_rate}
    best_params['metric'] = 'binary_logloss'

    lgb_model = LGBMClassifier(**best_params)

    lgb_model.fit(X_train, y_train)

    return lgb_model


# предикт LightGBM model с проверкой данных
def predict_lightgbm(
    candidates: pd.DataFrame,
    events: pd.DataFrame,
    lgb_model: LGBMClassifier
) -> pd.DataFrame:
    interactions_featurized = enrich_interactions(candidates, events)

    print('Running LightGBM scoring')
    interactions = interactions_featurized[['user_id', 'item_id']].copy()
    features = interactions_featurized.drop(columns=['user_id', 'item_id'])

    # Преобразование временных данных в числовой формат
    features = convert_timedelta_to_numeric(features)

    for col in features.columns:
        if features[col].dtype == 'object':
            features[col] = pd.to_numeric(features[col], errors='coerce')
            features[col].fillna(0, inplace=True)

    scores = lgb_model.predict_proba(features)[:, 1].flatten()
    interactions['lightgbm_score'] = scores
    return interactions


Шаманим с данными и ALS

In [19]:
# Объединение метаинформации о фильмах с транзакциями
def add_meta_to_events(events, item):
    events_with_meta = (
        events
        .merge(item,
               on='item_id', how='left')
    )
    # Пример вычисления отношения просмотров
    events_with_meta['watched_ratio'] = events_with_meta['rating'] / 5.0

    def score_event(row):
        score = 1
        if row['rating'] >= 3:  # Пример: учитываем рейтинг от 3 и выше
            score = int(row['rating'] * 2)
        return min(score, 10)

    events_with_meta['score'] = events_with_meta.apply(score_event, axis=1)
    return events_with_meta

# Добавление рейтингов к событиям
def add_ratings_to_events(events):
    events['score'].fillna(events['rating'], inplace=True)
    return events

# Кодирование TF-IDF
def encode_tfidf_coo(events: pd.DataFrame) -> pd.DataFrame:
    score_sum_per_user = (
        events
        .groupby('user_id')['score']
        .transform('sum')
    )
    user_count_per_item = (
        events
        .groupby('item_id')['user_id']
        .transform('size')
    )
    tf = events['score'].values / score_sum_per_user.values
    idf = np.log(len(score_sum_per_user) / user_count_per_item.values)

    tfidf = events[['user_id', 'item_id']].copy()
    tfidf['value'] = tf * idf

    return tfidf

# Преобразование данных в формат TF-IDF для ALS
def encode_tfidf(events: pd.DataFrame) -> Tuple[LabelEncoder, LabelEncoder, csr_matrix]:
    tfidf = encode_tfidf_coo(events)

    user_encoder = LabelEncoder()
    item_encoder = LabelEncoder()

    user_index = user_encoder.fit_transform(events['user_id'].values)
    item_index = item_encoder.fit_transform(events['item_id'].values)

    tfidf_csr = csr_matrix(
        (
            tfidf['value'].astype('float32').values,
            (user_index, item_index)
        ),
        shape=(len(user_encoder.classes_), len(item_encoder.classes_))
    )

    return user_encoder, item_encoder, tfidf_csr

# Функция для запуска ALS
def als_fit_predict(events_csr: csr_matrix):
    als = AlternatingLeastSquares(factors=128, iterations=30, alpha=40.0, calculate_training_loss=True)
    als.fit(events_csr)

    recommendations_matrix, recommendations_scores = als.recommend(
        np.arange(0, events_csr.shape[0]),
        events_csr,
        N=10,
        filter_already_liked_items=True
    )

    return recommendations_matrix, recommendations_scores

# Преобразование рекомендаций в DataFrame
def als_recommendations_to_df(
    recommendations_matrix: np.ndarray,
    recommendations_scores: np.ndarray,
    user_encoder: LabelEncoder,
    item_encoder: LabelEncoder,
    user_key = 'user_id',
    item_key = 'item_id'
) -> pd.DataFrame:
    recommendations_indices = pd.DataFrame({
        'user_index': np.arange(0, len(recommendations_matrix)),
        'item_index': list(recommendations_matrix),
        'score': list(recommendations_scores),
    })

    user_mapping = pd.DataFrame({
        'user_index': np.arange(0, len(user_encoder.classes_)),
        user_key: user_encoder.classes_,
    })

    item_mapping = pd.DataFrame({
        'item_index': np.arange(0, len(item_encoder.classes_)),
        item_key: item_encoder.classes_,
    })

    recommendations = (
        recommendations_indices
        .merge(
            user_mapping,
            on='user_index',
            how='left',
        )
        .drop(columns=['user_index'])
        .explode(['item_index', 'score'], ignore_index=True)
        .merge(
            item_mapping,
            on='item_index',
            how='left',
        )
        .drop(columns=['item_index'])
    )

    return recommendations

# Главная функция для запуска ALS
def run_als(events: pd.DataFrame, item: pd.DataFrame) -> pd.DataFrame:
    print('Preprocess events')
    events = add_meta_to_events(events, item)
    events = add_ratings_to_events(events)

    print('Compute TF-IDF')
    user_encoder, item_encoder, events_csr = encode_tfidf(events)

    print('Run ALS')
    recommendations_item_indices, recommendations_scores = als_fit_predict(events_csr)

    print('Postprocess ALS prediction')
    recommendations = als_recommendations_to_df(
        recommendations_item_indices,
        recommendations_scores,
        user_encoder,
        item_encoder,
        user_key='user_id',
        item_key='item_id',
    )

    return recommendations

# Выполнение рекомендаций
recommendations = run_als(events, item)
print(recommendations.head())

Preprocess events


<ipython-input-19-97e16ea0613f>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  events['score'].fillna(events['rating'], inplace=True)


Compute TF-IDF
Run ALS


  0%|          | 0/30 [00:00<?, ?it/s]

Postprocess ALS prediction
      score  user_id  item_id
0  0.990891        0     1811
1  0.972504        0     2256
2  0.741381        0     1545
3  0.722323        0      331
4   0.71254        0     2732


Еще несколько функций для обработки данных

In [20]:
# Функция для объединения новых фич с item_features
def merge_item_feature(item_features, new_feature):
    item_features = item_features.merge(
        new_feature,
        on='item_id',
        how='left'
    )
    return item_features

# Функция для объединения новых фич с user_features
def merge_user_feature(user_features, new_feature):
    user_features = user_features.merge(
        new_feature,
        on='user_id',
        how='left'
    )
    return user_features

# Добавление фичи популярности фильмов
def add_item_popularity_feature(item_features: pd.DataFrame, events: pd.DataFrame):
    item_occurrences = (
        events
        .groupby('item_id')
        .size()
        .reset_index(name='item_occurrences')
    )

    item_occurrences['item_popularity'] = (
        item_occurrences['item_occurrences'] /
        events['item_id'].nunique()
    )
    item_occurrences.drop(columns=['item_occurrences'], inplace=True)

    return merge_item_feature(item_features, item_occurrences)

# Добавление фичи числа закладок на фильмы (если применимо)
def add_item_bookmark_count_feature(item_features: pd.DataFrame, events: pd.DataFrame):
    bookmarks_per_item = (
        events
        .groupby('item_id')
        .size()
        .reset_index(name='item_bookmark_count')
    )

    item_features = merge_item_feature(item_features, bookmarks_per_item)
    item_features['item_bookmark_count'].fillna(0, inplace=True)

    return item_features

# Добавление фичи количества просмотров для пользователя
def add_user_watch_count_feature(user_features: pd.DataFrame, events: pd.DataFrame):
    user_watch_count = (
        events
        .groupby('user_id')
        .size()
        .reset_index(name='user_watch_count')
    )
    return merge_user_feature(user_features, user_watch_count)

# Добавление фичи стандартного отклонения времени просмотра для пользователя
def add_user_watch_time_std(user_features: pd.DataFrame, events: pd.DataFrame):
    user_watch_time_std = (
        events
        .groupby('user_id')['timestamp']
        .std()
        .reset_index(name='user_watch_time_std')
    )
    return merge_user_feature(user_features, user_watch_time_std)

# Генерация фич для фильмов
def generate_item_features(events: pd.DataFrame) -> pd.DataFrame:
    item_features = pd.DataFrame({
        'item_id': events['item_id'].unique()
    })
    item_features = add_item_popularity_feature(item_features, events)
    item_features = add_item_bookmark_count_feature(item_features, events)
    return item_features

# Убедитесь, что timestamp имеет тип datetime
events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')

# Добавление фичи количества просмотров за последнюю неделю для пользователей
def add_recent_watch_count(user_features: pd.DataFrame, events: pd.DataFrame):
    current_time = pd.Timestamp.now()  # Текущее время
    recent_watches = (
        events
        .loc[events['timestamp'] >= current_time - pd.Timedelta(weeks=1)]  # Изменение с использованием loc
        .groupby('user_id')
        .size()
        .reset_index(name='recent_watch_count')
    )
    user_features = merge_user_feature(user_features, recent_watches)
    user_features['recent_watch_count'].fillna(0, inplace=True)
    return user_features
# Добавление фичи средней оценки пользователя
def add_user_avg_rating(user_features: pd.DataFrame, events: pd.DataFrame):
    avg_rating = (
        events
        .groupby('user_id')['rating']
        .mean()
        .reset_index(name='user_avg_rating')
    )
    user_features = merge_user_feature(user_features, avg_rating)
    user_features['user_avg_rating'].fillna(0, inplace=True)
    return user_features

# Добавление этих фич в генерацию фич для пользователей
def generate_user_features(events: pd.DataFrame) -> pd.DataFrame:
    user_features = pd.DataFrame({
        'user_id': events['user_id'].unique()
    })
    user_features = add_user_watch_count_feature(user_features, events)
    user_features = add_user_watch_time_std(user_features, events)
    user_features = add_recent_watch_count(user_features, events)  # Новая фича
    user_features = add_user_avg_rating(user_features, events)  # Новая фича
    return user_features

# Обогащение взаимодействий (интеракций) фичами
def enrich_interactions(
    interactions: pd.DataFrame,
    events: pd.DataFrame
) -> pd.DataFrame:
    print('Generating item features')
    item_features = generate_item_features(events)

    print('Generating user features')
    user_features = generate_user_features(events)

    print('Merging features to interactions')
    interactions_featurized = (
        interactions
        .merge(
            item_features,
            on='item_id',
            how='left'
        )
        .merge(
            user_features,
            on='user_id',
            how='left'
        )
    )

    return interactions_featurized

# Функция для оценки интеракций с использованием CatBoost
def score_interactions(
    interactions_featurized: pd.DataFrame,
    catboost_model: CatBoostClassifier,
) -> pd.DataFrame:
    interactions = interactions_featurized[['user_id', 'item_id']].copy()
    features = interactions_featurized.drop(columns=['user_id', 'item_id'])
    scores = catboost_model.predict_proba(features)[:, 1].flatten()
    interactions['catboost_score'] = scores
    return interactions

Переходим к catboost

In [21]:


# Обучение модели CatBoost
def fit_catboost(
    candidates: pd.DataFrame,
    test_events: pd.DataFrame,
    events: pd.DataFrame
):
    print('Labeling candidates')
    candidates_labeled = (
        candidates
        .merge(
            (
                test_events[['user_id', 'item_id']]
                .assign(label=1)
            ),
            on=['user_id', 'item_id'],
            how='left',
        )
    )

    candidates_labeled['label'] = (
        candidates_labeled['label']
        .fillna(0)
        .astype('int32')
    )

    positive_classes_rate = candidates_labeled['label'].sum() / len(candidates_labeled)
    print(f'Positive classes = {positive_classes_rate * 100:.2f}%')

    print('Extracting features for candidates')
    cb_features = enrich_interactions(candidates_labeled, events)
    del candidates_labeled

    cb_pool = Pool(
        cb_features.drop(columns=['user_id', 'item_id', 'label']),
        cb_features['label']
    )
    del cb_features

    print('Training CatBoost')
    cb_cls = CatBoostClassifier(
        iterations=100,
        class_weights=[positive_classes_rate, 1 - positive_classes_rate],
        eval_metric='BalancedAccuracy',
        objective='Logloss'
    )

    cb_cls.fit(cb_pool, verbose=1)

    return cb_cls

# Предсказание с использованием обученной модели CatBoost
def predict_catboost(
    candidates: pd.DataFrame,
    events: pd.DataFrame,
    catboost_model: CatBoostClassifier
) -> pd.DataFrame:
    interactions_featurized = enrich_interactions(candidates, events)

    print('Running CatBoost scoring')
    scored_interactions = score_interactions(interactions_featurized, catboost_model)

    return scored_interactions


Переходим к обучению

In [22]:

# Добавляем ранжирование событий для каждого пользователя по времени
events['split_rank_per_user'] = (
    events
    .groupby('user_id')['timestamp']
    .rank('first', ascending=False)
    .astype('int32')
)

# Разделение данных на тренировочный и тестовый наборы для ALS
als_events_train = (
    events
    .query('split_rank_per_user > 2')
    .drop(columns=['split_rank_per_user'])
)

als_events_test = (
    events
    .query('split_rank_per_user <= 2')
    .drop(columns=['split_rank_per_user'])
)[["item_id", "user_id"]]

# Запуск ALS для создания кандидатов для обучения модели CatBoost
als_candidates_to_train_catboost = run_als(als_events_train, item)

# Обучение модели CatBoost с использованием кандидатов, сгенерированных ALS
catboost_model = fit_catboost(
    als_candidates_to_train_catboost,
    als_events_test,
    events
)

# Training LightGBM
lightgbm_model = fit_lightgbm(
    als_candidates_to_train_catboost,
    als_events_test,
    events
)

# Запуск ALS на всех данных для генерации кандидатов
als_candidates = run_als(events, item)

# Prediction with CatBoost
catboost_prediction_full = predict_catboost(als_candidates, events, catboost_model)

# Prediction with LightGBM
lightgbm_prediction_full = predict_lightgbm(als_candidates, events, lightgbm_model)

# Combining predictions
combined_predictions = (
    catboost_prediction_full[['user_id', 'item_id', 'catboost_score']]
    .merge(
        lightgbm_prediction_full[['user_id', 'item_id', 'lightgbm_score']],
        on=['user_id', 'item_id'],
        how='inner'
    )
)

# Ensemble by averaging the scores
combined_predictions['final_score'] = (
    combined_predictions['catboost_score'] * 0.5 +
    combined_predictions['lightgbm_score'] * 0.5
)

# Select the top 10 recommendations per user based on the final ensemble score
final_recommendations = (
    combined_predictions
    .sort_values('final_score', ascending=False, ignore_index=True)
    .groupby('user_id')
    .head(10)
)

Preprocess events


<ipython-input-19-97e16ea0613f>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  events['score'].fillna(events['rating'], inplace=True)


Compute TF-IDF
Run ALS


  0%|          | 0/30 [00:00<?, ?it/s]

Postprocess ALS prediction
Labeling candidates
Positive classes = 1.45%
Extracting features for candidates
Generating item features
Generating user features
Merging features to interactions


...

  0%|          | 0/30 [00:00<?, ?it/s]

Postprocess ALS prediction
Generating item features
Generating user features
Merging features to interactions
Running CatBoost scoring


<ipython-input-20-5736fd34f786>:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  item_features['item_bookmark_count'].fillna(0, inplace=True)
<ipython-input-20-5736fd34f786>:93: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

Generating item features
Generating user features
Merging features to interactions
Running LightGBM scoring


<ipython-input-18-ae045af3a25a>:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features[col].fillna(0, inplace=True)


Получаем наш сабмишен

In [23]:
# Создание DataFrame для сабмита
submission = final_recommendations[['user_id', 'item_id']].copy()

# Группировка по user_id и объединение item_id в строку, разделенную пробелами
submission = (
    submission
    .groupby('user_id')['item_id']
    .apply(lambda x: ' '.join(map(str, x)))
    .reset_index()
)

# Сохранение сабмита в CSV файл
submission.to_csv('submission_bobs.csv', index=False)

In [24]:
submission

,user_id,item_id
0,0,785 1001 2732 760 1545 2256 3463 1248 1811 331
1,1,3205 106 232 3656 2175 1491 1246 1801 1039 1686
2,2,1781 1809 2774 2185 2639 2338 3005 452 2342 2354
3,3,2194 3318 3528 2186 1861 3472 3435 183 1617 3327
4,4,487 37 2076 2774 1858 1809 3677 188 1337 1191
...,...,...
6035,6035,1855 1011 113 717 2366 1375 1216 1374 1403 3013
6036,6036,1379 2798 3473 3013 494 3153 3142 772 2370 2297
6037,6037,1968 1840 1775 3212 2603 618 2335 2784 3583 1747
6038,6038,1289 1808 2626 1100 361 1070 3105 2587 182 2920
